# Complete Simulation Chain for Visibility Calculation

## Read in Config

In [23]:
import vipy.simulation.utils as ut
import vipy.layouts.layouts as layouts
import astropy.constants as const
from astropy import units as un
import time as t

In [24]:
rc = ut.read_config('/net/nfshome/home/sfroese/vipy/config/default.toml')
array_layout = layouts.get_array_layout('eht')

src_crd = rc['src_coord']
wave1 = const.c/((float(rc['channel'].split(':')[0])-float(rc['channel'].split(':')[1]))*10**6/un.second)/un.meter
wave2 = const.c/((float(rc['channel'].split(':')[0])+float(rc['channel'].split(':')[1]))*10**6/un.second)/un.meter

## Generate (l,m)-plane / FOV

In [25]:
import vipy.simulation.scan as scan

In [26]:
grid = scan.create_bgrid(rc['fov_size'], 256, src_crd)
lm = scan.lm(grid, src_crd)

## Calculate Start and Stop times for every measurement

In [27]:
time = ut.calc_time_steps(rc)

## Calculate Baselines for one measurement

In [28]:
baselines = scan.get_baselines(src_crd, time[(0):(31)], array_layout)

## Calculate Mueller/Jones matrices

In [29]:
import numpy as np
import torch

In [30]:
%%time
start = t.time()
torch.set_num_threads(8)
JJ_f1 = scan.getJones(lm, baselines, wave1, time, src_crd, array_layout)
JJ_f2 = scan.getJones(lm, baselines, wave2, time, src_crd, array_layout)

CPU times: user 25 s, sys: 14.4 s, total: 39.4 s
Wall time: 5.67 s


## Open Source Image and compute Stokes vector

In [31]:
from astropy.io import fits
import matplotlib.pyplot as plt
import torch

In [32]:
hdul = fits.open('celestial-03-05.fits')
img = hdul[0].data.astype(np.float32)
img = torch.tensor(img)

I = torch.zeros((img.shape[0],img.shape[1],4), dtype=torch.cdouble)
I[...,0] = img
I[...,1] = img*torch.sqrt(torch.tensor(0.5))
I[...,2] = img*torch.sqrt(torch.tensor(0.5))
# plt.imshow((I[...,1]+I[...,2]).real)
# plt.colorbar()

## Integration

In [33]:
%%time
delta_t = rc['corr_int_time']
delta_f = float(rc['channel'].split(':')[1])*10**6
delta_l = lm[255,0,0]-lm[0,0,0]
delta_m = lm[0,255,0]-lm[0,0,0]
print(I.shape)
print(JJ_f1.shape)

integral = scan.integrate(JJ_f1, JJ_f2, I, 28, delta_t, delta_f, delta_l, delta_m)
integral.shape
end = t.time()
print(end - start)

torch.Size([256, 256, 4])
torch.Size([256, 256, 186, 4, 4])
7.279691219329834
CPU times: user 7.33 s, sys: 5.3 s, total: 12.6 s
Wall time: 1.58 s


In [12]:
integral

tensor([[-1.1334e-03-3.9918e-04j, -6.5493e-04-2.8166e-04j,
         -2.3607e-03-3.3852e-04j, -1.2898e-03-2.1407e-04j],
        [ 2.7617e-04+1.2189e-04j,  1.3868e-04+5.5854e-05j,
          1.1486e-03+5.6955e-04j,  5.4267e-04+2.3269e-04j],
        [-6.1208e-05-3.9296e-05j, -1.4727e-04-1.2537e-04j,
         -2.6537e-04-1.1459e-04j, -6.0833e-04-4.7192e-04j],
        [ 1.6390e-04+2.4865e-04j,  9.2562e-05+1.4124e-04j,
          3.4096e-04+4.2753e-04j,  1.8158e-04+2.2953e-04j],
        [ 1.7229e-03-1.3486e-03j,  4.5350e-04-6.1742e-04j,
          1.3357e-03-1.0136e-03j,  3.2809e-04-3.7811e-04j],
        [ 8.0898e-04+8.2825e-04j,  1.8717e-03-3.9629e-04j,
          4.7735e-04+6.1450e-04j,  9.2879e-04-1.4229e-04j],
        [ 4.4335e-04-1.2746e-04j,  2.9454e-04-3.3192e-05j,
          1.0848e-03-4.8307e-04j,  6.8600e-04-2.5204e-04j],
        [-2.1826e-05+2.3613e-04j, -1.2098e-05+1.4243e-04j,
         -9.4203e-05+1.0061e-03j, -5.1801e-05+6.0602e-04j],
        [ 2.5197e-04+6.3523e-04j,  1.3901e-04+1.

## All scan loop

In [34]:
from tqdm import tqdm

In [35]:
torch.set_num_threads(48)

In [36]:
hdul = fits.open('celestial-03-05.fits')
img = hdul[0].data.astype(np.float32)
img = torch.tensor(img)

I = torch.zeros((img.shape[0],img.shape[1],4), dtype=torch.cdouble)
I[...,0] = img
I[...,1] = img*torch.sqrt(torch.tensor(0.5))
I[...,2] = img*torch.sqrt(torch.tensor(0.5))

In [37]:
integral2 = np.array([])
for i in tqdm(range(72)):
    t = time[i*30:(i+1)*30+1]
    baselines = scan.get_baselines(src_crd, t, array_layout)
    
    JJ_f1 = scan.getJones(lm, baselines, wave1, time, src_crd, array_layout)
    if JJ_f1.shape[0] == 1:
        continue
    JJ_f2 = scan.getJones(lm, baselines, wave2, time, src_crd, array_layout)
    delta_t = rc['corr_int_time']
    delta_f = float(rc['channel'].split(':')[1])*10**6
    delta_l = lm[255,0,0]-lm[0,0,0]
    delta_m = lm[0,255,0]-lm[0,0,0]

    integral2 = np.append(integral2,scan.integrate(JJ_f1, JJ_f2, I, 28, delta_t, delta_f, delta_l, delta_m))


100%|██████████| 72/72 [06:12<00:00,  5.17s/it]


In [38]:
integral2.reshape(-1, 4)

array([[-1.13336190e-03-3.99177127e-04j, -6.54927352e-04-2.81659092e-04j,
        -2.36066731e-03-3.38521249e-04j, -1.28975794e-03-2.14071903e-04j],
       [ 2.76169261e-04+1.21888920e-04j,  1.38684767e-04+5.58543057e-05j,
         1.14862817e-03+5.69553970e-04j,  5.42665164e-04+2.32686642e-04j],
       [-6.12077257e-05-3.92960242e-05j, -1.47273721e-04-1.25366179e-04j,
        -2.65366234e-04-1.14587693e-04j, -6.08326377e-04-4.71917318e-04j],
       ...,
       [ 2.03031445e-03+8.38618388e-04j,  1.76866397e-04+7.64811840e-05j,
        -1.23450025e-03+2.87408881e-05j, -1.05803780e-04+4.60967566e-05j],
       [ 3.22056769e-05+6.09329581e-04j, -9.43747788e-05-6.15605610e-05j,
        -1.90338560e-04-4.63722330e-05j,  2.71498065e-05+9.21930513e-05j],
       [-2.02488871e-03-6.61459376e-04j, -3.49381229e-04-1.16094085e-04j,
         9.27046547e-05-3.92226240e-04j, -2.57301956e-05-4.38029391e-05j]])